In [1]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
import re
from bs4 import BeautifulSoup
import pandas as pd
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier,  AdaBoostClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from xgboost.sklearn import XGBClassifier

In [2]:
def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

def tokenizer_stem(text):
    porter = PorterStemmer()
    return [porter.stem(word) for word in re.split('\s+', text.strip())]

nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

def get_stream(path, size):
    for chunk in pd.read_csv(path, chunksize=size):
        yield chunk

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jinghao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def preprocessor_author(text):
    ts = BeautifulSoup(text, 'html.parser').find_all( attrs={"class": "author_name"})
    #print(ts)
    text = ""
    for t in ts:
        text += t.get_text()+ " "
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    # print(text)
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

def preprocessor_title(text):
    ts = BeautifulSoup(text, 'html.parser').find_all( attrs={"class": "title"})
    #print(ts)
    text = ""
    for t in ts:
        text += t.get_text()+ " "
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    # print(text)
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

# def preprocessor_paragraph(text):
#     ts = BeautifulSoup(text, 'html.parser').find_all('p')
#     #print(ts)
#     text = ""
#     for t in ts:
#         text += t.get_text()+ " "
#     r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
#     emoticons = re.findall(r, text)
#     text = re.sub(r, '', text)
# #     print(text)
#     # convert to lowercase and append all emoticons behind (with space in between)
#     # replace('-','') removes nose of emoticons
#     text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
#     return text

def preprocessor_time(text):
    ts = BeautifulSoup(text, 'html.parser').find_all('time')
    #print(ts)
    text = ""
    for t in ts:
        text += t.get_text()+ " "
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
#     print(text)
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

def preprocessor_href(text_):
    soup = BeautifulSoup(text_)
    
    text = ''
    for link in soup.find_all('a'):
        t = link.get('href')
        if t != None:
            if t.find('https://') != -1:
                text += t.replace("https://", "").split('/')[0].replace("www", "").replace("com", "").replace(".", " ")
                
            if t.find('http://') != -1:
                text += t.replace("http://", "").split('/')[0].replace("www", "").replace("com", "").replace(".", " ")
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

# def preprocessor_channel(text):
#     ts = BeautifulSoup(text, 'html.parser').find_all('article')
#     # print(ts)
#     text = ""
#     for t in ts:
#         text += t.get_text()+ " "
#     r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
#     emoticons = re.findall(r, text)
#     text = re.sub(r, '', text)
#     print(text)
#     # convert to lowercase and append all emoticons behind (with space in between)
#     # replace('-','') removes nose of emoticons
#     text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
#     return text

In [8]:
from scipy.sparse import hstack
def partial_pipe_fit(pipeline_obj, X, Y):
    feature = pipeline_obj.named_steps['vect'].fit_transform(X)
    print("Before reduce dim")
    print(feature.shape)
    print(Y.shape)
    feature = pipeline_obj.named_steps['select'].fit_transform(feature, Y)
    print("After reduce dim")
    print(feature.shape)
    print(Y.shape)
    # concate_features = np.hstack([concate_features, feature])
    # concate_features = np.concatenate((feature, concate_features), axis=1)
    return feature
def partial_pipe_predict(pipeline_obj, X):
    feature = pipeline_obj.named_steps['vect'].fit_transform(X)
    feature = pipeline_obj.named_steps['select'].transform(feature)
    return feature
class stacking_classifier():
    def __init__(self):
        # pipelines for estimators
        self.pipes = []
        self.pipes.append(Pipeline([('vect', HashingVectorizer(n_features=2**20, preprocessor=preprocessor_title, tokenizer=tokenizer_stem_nostop, alternate_sign=False)), 
                  ('select', SelectKBest(chi2, k=50))]))
        self.pipes.append( Pipeline([('vect', HashingVectorizer(n_features=2**20, preprocessor=preprocessor_author, tokenizer=tokenizer_stem_nostop, alternate_sign=False)), 
                  ('select', SelectKBest(chi2, k=50))]))
        self.pipes.append( Pipeline([('vect', HashingVectorizer(n_features=2**20, preprocessor=preprocessor_time, tokenizer=tokenizer_stem_nostop, alternate_sign=False)), 
                  ('select', SelectKBest(chi2, k=50))]))
        self.pipes.append( Pipeline([('vect', HashingVectorizer(n_features=2**20, preprocessor=preprocessor_href, tokenizer=tokenizer_stem_nostop, alternate_sign=False)), 
                  ('select', SelectKBest(chi2, k=50))]))
        self.concate_features = []
        self.concate_predict_features = []
        
        self.final_est = XGBClassifier(n_estimators=100, learning_rate= 0.3, max_depth=7)
    def fit(self, X, Y, classes):
        self.concate_features = []
        i = 0
        for p in self.pipes:
            # print(i)
            i+=1
            self.concate_features.append(partial_pipe_fit(p, X, Y))
            # print('feature shape', self.concate_features[-1].shape)
        # print(self.concate_features)
        self.feature_final = hstack(self.concate_features)
        # print(self.feature_final.shape)
        self.final_est.fit(self.feature_final, Y)

    def predict_proba(self, X):
        self.concate_features = []
        for p in self.pipes:
            self.concate_features.append(partial_pipe_predict(p, X))
            # print('feature shape', self.concate_features[-1].shape)
        # print(self.concate_features)
        self.feature_final = hstack(self.concate_features)
        # print(self.feature_final.shape)
        return self.final_est.predict_proba(self.feature_final)

In [10]:
# def partial_pipe_fit(pipeline_obj, X, Y, classes):
#     feature = pipeline_obj.named_steps['vect'].fit_transform(X)
#     print(feature.shape)
#     print(Y.shape)
#     feature = pipeline_obj.named_steps['select'].fit_transform(feature, Y)
#     print(feature.shape)
#     print(Y.shape)
#     pipeline_obj.named_steps['clf'].fit(feature, Y)
# def partial_pipe_predict(pipeline_obj, X):
#     feature = pipeline_obj.named_steps['vect'].fit_transform(X)
#     feature = pipeline_obj.named_steps['select'].transform(feature)
#     return pipeline_obj.named_steps['clf'].predict_proba(feature)
# class stacking_classifier():
#     def __init__(self):
#         # pipelines for estimators
#         self.pipes = []
#         self.pipes.append(Pipeline([('vect', HashingVectorizer(n_features=2**15, preprocessor=preprocessor_title, tokenizer=tokenizer_stem_nostop, alternate_sign=False)), 
#                   ('select', SelectKBest(chi2, k=50)),
#                   ('clf',XGBClassifier(n_estimators=100, learning_rate= 0.3, max_depth=5))]))
#         self.pipes.append( Pipeline([('vect', HashingVectorizer(n_features=2**15, preprocessor=preprocessor_author, tokenizer=tokenizer_stem_nostop, alternate_sign=False)), 
#                   ('select', SelectKBest(chi2, k=50)),
#                   ('clf',XGBClassifier(n_estimators=100, learning_rate= 0.3, max_depth=5))]))
#         self.pipes.append( Pipeline([('vect', HashingVectorizer(n_features=2**15, preprocessor=preprocessor_time, tokenizer=tokenizer_stem_nostop, alternate_sign=False)), 
#                   ('select', SelectKBest(chi2, k=50)),
#                   ('clf',XGBClassifier(n_estimators=100, learning_rate= 0.3, max_depth=5))]))
#         self.pipes.append( Pipeline([('vect', HashingVectorizer(n_features=2**15, preprocessor=preprocessor_href, tokenizer=tokenizer_stem_nostop, alternate_sign=False)), 
#                   ('select', SelectKBest(chi2, k=50)),
#                   ('clf',XGBClassifier(n_estimators=100, learning_rate= 0.3, max_depth=5))]))
        
#         self.final_est = XGBClassifier(n_estimators=100, learning_rate= 0.3, max_depth=5)
#         # self.pipes.append(Pipeline([('vect', HashingVectorizer(n_features=2**20, preprocessor=preprocessor_paragraph, tokenizer=tokenizer_stem_nostop, alternate_sign=False)), 
#         #           ('select', SelectKBest(chi2, k=500)),
#         #           ('clf',RandomForestClassifier(criterion='entropy', n_estimators=200, random_state=1, n_jobs=2, max_depth=8))]))
#         # self.pipes.append( Pipeline([('vect', HashingVectorizer(n_features=2**20, preprocessor=preprocessor_author, tokenizer=tokenizer_stem_nostop, alternate_sign=False)), 
#         #           ('select', SelectKBest(chi2, k=100)),
#         #           ('clf',RandomForestClassifier(criterion='entropy', n_estimators=200, random_state=1, n_jobs=2, max_depth=8))]))
#         # self.pipes.append( Pipeline([('vect', HashingVectorizer(n_features=2**20, preprocessor=preprocessor_time, tokenizer=tokenizer_stem_nostop, alternate_sign=False)), 
#         #           ('select', SelectKBest(chi2, k=7)),
#         #           ('clf',RandomForestClassifier(criterion='entropy', n_estimators=200, random_state=1, n_jobs=2, max_depth=8))]))
        
#         # self.final_est = RandomForestClassifier(criterion='entropy', n_estimators=200, random_state=1, n_jobs=2, max_depth=8)
#     def fit(self, X, Y, classes):
#         self.y_preds = []
#         for p in self.pipes:
#             partial_pipe_fit(p, X_train, y_train, classes=classes)
#             self.y_preds.append(partial_pipe_predict(p, X))
        
#         self.ys = self.y_preds[0]
#         for i in range(1, len(self.y_preds)):
#             self.ys = np.concatenate((self.ys, self.y_preds[i]), axis=1)
        
#         self.final_est.fit(self.ys, Y)
#     def predict_proba(self, X):
#         self.y_preds = []
#         for p in self.pipes:
#             self.y_preds.append(partial_pipe_predict(p, X))
#         self.ys = self.y_preds[0]
#         for i in range(1, len(self.y_preds)):
#             self.ys = np.concatenate((self.ys, self.y_preds[i]), axis=1)
#         return self.final_est.predict_proba(self.ys)

In [9]:
# clf = XGBClassifier(n_estimators=100, learning_rate= 0.1, max_depth=5, objective='binary:logistic', eval_metric='error')
stream = get_stream(path='/home/jinghao/miniconda3/DL_comp1/train.csv', size=23000)
classes = np.array([-1, 1])
batch = next(stream)
X_train, y_train = batch['Page content'], batch['Popularity']
# print(X_train)
model = stacking_classifier()
train_auc, val_auc = [], []
model.fit(X_train, y_train, classes)

score = roc_auc_score(y_train, model.predict_proba(X_train)[:,1])
train_auc.append(score)
print('train: {}'.format(score))

/home/jinghao/miniconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Before reduce dim
(23000, 1048576)
(23000,)
After reduce dim
(23000, 50)
(23000,)


/home/jinghao/miniconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Before reduce dim
(23000, 1048576)
(23000,)
After reduce dim
(23000, 50)
(23000,)


/home/jinghao/miniconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Before reduce dim
(23000, 1048576)
(23000,)
After reduce dim
(23000, 50)
(23000,)


/home/jinghao/miniconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Before reduce dim
(23000, 1048576)
(23000,)
After reduce dim
(23000, 50)
(23000,)
[11:55:05] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/jinghao/miniconda3/envs/pytorch/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jinghao/miniconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


train: 0.6462946873614254


In [10]:
# validate
def get_stream_from_x(path, size, specific_x):
    count = 0
    chunk_list = []
    for chunk in pd.read_csv(path, chunksize=1000, iterator=True):
        if(count < specific_x):
            print(count)
            count+=1
        else:
            chunk_list.append(chunk)
    df = pd.concat(chunk_list)
    return df

batch = get_stream_from_x(path='/home/jinghao/miniconda3/DL_comp1/train.csv', size=1000, specific_x=23)
X_val, y = batch['Page content'], batch['Popularity']
print(X_val)

score = roc_auc_score(y, model.predict_proba(X_val)[:,1])

val_auc.append(score)
print('validation: {}'.format(score))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23000    <html><head><div class="article-info"><span cl...
23001    <html><head><div class="article-info"><span cl...
23002    <html><head><div class="article-info"><span cl...
23003    <html><head><div class="article-info"><span cl...
23004    <html><head><div class="article-info"><span cl...
                               ...                        
27638    <html><head><div class="article-info"><span cl...
27639    <html><head><div class="article-info"><span cl...
27640    <html><head><div class="article-info"><span cl...
27641    <html><head><div class="article-info"><span cl...
27642    <html><head><div class="article-info"><span cl...
Name: Page content, Length: 4643, dtype: object


/home/jinghao/miniconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


validation: 0.5371189683520579


In [42]:
# import _pickle as pkl
# # import optimized pickle written in C for serializing and 
# # de-serializing a Python object

# # dump to disk
# pkl.dump(model, open('output/model_xgb.pkl', 'wb'))

In [44]:
# import _pickle as pkl
# model = pkl.load(open('output/model_xgb.pkl', 'rb'))

In [11]:
df_test = pd.read_csv('/home/jinghao/miniconda3/DL_comp1/test.csv')
df_test_pred = pd.DataFrame(columns=['Id', 'Popularity'])
# print(df_test)
for index, r in df_test.iterrows():
    pred = model.predict_proba([r['Page content']])[0][1]
    
    print(r['Id'], pred)
    df_test_pred = df_test_pred.append({'Id': int(r['Id']), 'Popularity':pred}, ignore_index=True)
df_test_pred['Id'] = df_test_pred['Id'].astype('int')
df_test_pred.to_csv("./xgb.csv", index=False)

/home/jinghao/miniconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


27643 0.44239178
27644 0.6450865
27645 0.47223702
27646 0.47223702
27647 0.47223702
27648 0.7246223
27649 0.41186947
27650 0.80140334
27651 0.47223702
27652 0.4985988
27653 0.47223702
27654 0.5626866
27655 0.47223702
27656 0.36396712
27657 0.47223702
27658 0.5285076
27659 0.59094197
27660 0.47223702
27661 0.47223702
27662 0.47223702
27663 0.37534815
27664 0.47388017
27665 0.47223702
27666 0.4700679
27667 0.51500356
27668 0.47223702
27669 0.5277834
27670 0.83141375
27671 0.47223702
27672 0.47223702
27673 0.47223702
27674 0.48580208
27675 0.47223702
27676 0.6107023
27677 0.47223702
27678 0.47223702
27679 0.47223702
27680 0.4730325
27681 0.47388017
27682 0.72937036
27683 0.54797703
27684 0.4155967
27685 0.6133469
27686 0.47223702
27687 0.22884649
27688 0.47223702
27689 0.47223702
27690 0.5832949
27691 0.47223702
27692 0.47223702
27693 0.3520082
27694 0.47223702
27695 0.47223702
27696 0.47223702
27697 0.4451595
27698 0.47223702
27699 0.47223702
27700 0.47223702
27701 0.5285076
27702 0.4722

In [ ]:
# batch_size = 1000
# stream = get_stream(path='/home/jinghao/miniconda3/DL_comp1/train.csv', size=batch_size)
# classes = np.array([-1, 1])
# train_auc, val_auc = [], []
# model = stacking_classifier()
# # we use one batch for training and another for validation in each iteration
# iters = int((25000+batch_size-1)/(batch_size*2))
# for i in range(iters):
#     batch = next(stream)
#     X_train, y_train = batch['Page content'], batch['Popularity']
#     if X_train is None:
#         break
    
#     model.fit(X_train, y_train, classes)
    
#     score = roc_auc_score(y_train, model.predict_proba(X_train)[:,1])
#     train_auc.append(score)
#     print('train: [{}/{}] {}'.format((i+1)*(batch_size*2), 25000, score))

#     # validate
#     batch = next(stream)
#     X_val, y = batch['Page content'], batch['Popularity']

#     score = roc_auc_score(y, model.predict_proba(X_val)[:,1])

#     val_auc.append(score)
#     print('validation: [{}/{}] {}'.format((i+1)*(batch_size*2), 25000, score))

In [ ]:
# def pipe_fit(pipeline_obj, X, Y, classes):
#     pipeline_obj.named_steps['clf'].fit(pipeline_obj.named_steps['vect'].fit_transform(X),Y, classes)
# def pipe_predict(pipeline_obj, X):
#     return pipeline_obj.named_steps['clf'].predict_proba(pipeline_obj.named_steps['vect'].transform(X))
# class stacking_xgbclassifier():
#     def __init__(self):
#         # pipelines for estimators
#         self.pipes = []
#         self.pipes.append(Pipeline([('vect', HashingVectorizer(n_features=2**20, 
#                             preprocessor=preprocessor_tag, tokenizer=tokenizer_stem_nostop)), 
#                   ('clf',XGBClassifier(n_estimators=100, learning_rate= 0.1, max_depth=5, objective='binary:logistic', eval_metric='error'))]))
#         self.pipes.append( Pipeline([('vect', HashingVectorizer(n_features=2**20, 
#                             preprocessor=preprocessor, tokenizer=tokenizer_stem_nostop)), 
#                   ('clf',XGBClassifier(n_estimators=100, learning_rate= 0.1, max_depth=5, objective='binary:logistic', eval_metric='error'))]))
        
#         self.final_est = XGBClassifier(n_estimators=100, learning_rate= 0.1, max_depth=5, objective='binary:logistic', eval_metric='error')
#     def fit(self, X, Y, classes):
#         self.y_preds = []
#         for p in self.pipes:
#             pipe_fit(p, X_train, y_train, classes=classes)
#             self.y_preds.append(pipe_predict(p, X))
        
#         self.ys = self.y_preds[0]
#         for i in range(1, len(self.y_preds)):
#             self.ys = np.concatenate((self.ys, self.y_preds[i]), axis=1)
        
#         self.final_est.fit(self.ys, Y, classes=classes)
#     # def predict(self, X):
#     #     self.y_preds = []
#     #     for p in self.pipes:
#     #         self.y_preds.append(partial_pipe_predict(p, X))
#     #     self.ys = self.y_preds[0]
#     #     for i in range(1, len(self.y_preds)):
#     #         self.ys = np.concatenate((self.ys, self.y_preds[i]), axis=1)
#     #     return self.final_est.predict(self.ys)
#     def predict_proba(self, X):
#         self.y_preds = []
#         for p in self.pipes:
#             self.y_preds.append(pipe_predict(p, X))
#         self.ys = self.y_preds[0]
#         for i in range(1, len(self.y_preds)):
#             self.ys = np.concatenate((self.ys, self.y_preds[i]), axis=1)
#         return self.final_est.predict_proba(self.ys)

In [8]:
# L = [np.ones((3,2)), np.zeros((3,3)), np.ones((3,2))]
# np.concatenate(L, 1)

array([[1., 1., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 1., 1.]])